访问本地ollama模型

In [1]:
from langchain.llms import Ollama

# 选择 LLM，如 "mistral" 或 "llama2"
llm = Ollama(model="deepseek-r1:14b", base_url="http://localhost:11434")

# 生成文本
response = llm("你是谁")
print(response)


/var/folders/d7/yk13hmcn4233rjfm1pxqrv8m0000gn/T/ipykernel_47222/3506572564.py:4: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="deepseek-r1:14b", base_url="http://localhost:11434")
/var/folders/d7/yk13hmcn4233rjfm1pxqrv8m0000gn/T/ipykernel_47222/3506572564.py:7: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = llm("你是谁")


<think>

</think>

您好！我是由中国的深度求索（DeepSeek）公司开发的智能助手DeepSeek-R1。如您有任何任何问题，我会尽我所能为您提供帮助。


访问本地ollama模型+资料库

In [2]:
#公共结构定义

from langchain.document_loaders import (
    TextLoader,
    PyPDFLoader,
    CSVLoader,
    JSONLoader,
    UnstructuredMarkdownLoader
)
from langchain.embeddings import OllamaEmbeddings
import numpy as np

# 根据文档类型选择参数
splitter_configs = {
        "general": {
            "chunk_size": 1000,
            "chunk_overlap": 200
        },
        "technical": {
            "chunk_size": 1500,
            "chunk_overlap": 300,
            "separator": "\n\n"
        },
        "code": {
            "chunk_size": 800,
            "chunk_overlap": 100,
            "separator": "\n\n"
        },
        "markdown": {
            "chunk_size": 1000,
            "chunk_overlap": 150,
            "separator": "\n## "    # Markdown 二级标题作为分隔符
        },
        "news": {
            "chunk_size": 500,      # 减小块大小
            "chunk_overlap": 50,     # 适当的重叠
            "separator": "\n"        # 使用换行符作为分隔符
        }
    }

# 文件类型到加载器的映射
LOADER_MAPPING = {
    '.txt': TextLoader,
    '.md': UnstructuredMarkdownLoader,
    '.pdf': PyPDFLoader,
    '.csv': CSVLoader,
    '.json': JSONLoader
}

class NormalizedOllamaEmbeddings(OllamaEmbeddings):
    def embed_documents(self, texts):
        embeddings = super().embed_documents(texts)
        # L2 归一化
        normalized = [embedding/np.linalg.norm(embedding) for embedding in embeddings]
        return normalized
    
    def embed_query(self, text):
        embedding = super().embed_query(text)
        # L2 归一化
        return embedding/np.linalg.norm(embedding)


In [3]:
import os
from langchain.vectorstores import FAISS
from langchain.embeddings import OllamaEmbeddings
from langchain.document_loaders import (
    TextLoader,
    PyPDFLoader,
    CSVLoader,
    JSONLoader,
    UnstructuredMarkdownLoader
)
from langchain.text_splitter import CharacterTextSplitter
from typing import List, Dict

def load_documents_from_directory(
    directory_path: str,
    db_path: str,
    doc_type: str = "news",
    supported_extensions: List[str] = ['.txt', '.md', '.py', '.pdf']
) -> None:
    # 设置嵌入模型
    embedding = NormalizedOllamaEmbeddings(
        model="bge-m3:latest",
        base_url="http://localhost:11434"
    )
    
    # 获取分割器配置
    splitter_config = splitter_configs.get(doc_type, splitter_configs["general"])
    text_splitter = CharacterTextSplitter(**splitter_config)
    
    # 存储所有文档
    all_documents = []
    file_count = 0
    
    try:
        # 遍历目录
        for root, _, files in os.walk(directory_path):
            for file in files:
                if any(file.endswith(ext) for ext in supported_extensions):
                    file_path = os.path.join(root, file)
                    try:
                        # 添加调试信息
                        print(f"\n开始处理文件: {file_path}")
                        print(f"文件大小: {os.path.getsize(file_path)} bytes")
                        
                        # 尝试直接读取文件内容
                        # with open(file_path, 'r', encoding='utf-8') as f:
                        #     raw_content = f.read()
                        #     print(f"原始文件内容长度: {len(raw_content)} 字符")
                        
                        # 加载文档
                        file_extension = os.path.splitext(file_path)[1].lower()
                        if file_extension in LOADER_MAPPING:
                            loader_class = LOADER_MAPPING[file_extension]
                            loader = loader_class(file_path)  # 显式指定编码
                            documents = loader.load()
                            print(f"加载后的文档数量: {len(documents)}")
                            
                            if len(documents) == 0:
                                print(f"警告: 文档加载后为空")
                                continue
                                
                            # 分割文档
                            texts = text_splitter.split_documents(documents)
                            print(f"分割后的文本块数量: {len(texts)}")
                            
                            if len(texts) == 0:
                                print(f"警告: 文本分割后为空")
                                continue
                                
                            all_documents.extend(texts)
                            file_count += 1
                            print(f"成功加载文件: {file_path}")
                            
                    except Exception as e:
                        print(f"处理文件 {file_path} 时出错: {str(e)}")
                        print(f"错误类型: {type(e).__name__}")
                        import traceback
                        print(f"详细错误信息: {traceback.format_exc()}")
                        
        print(f"\n总共处理了 {file_count} 个文件")
        print(f"总共获得 {len(all_documents)} 个文本块")
        
        # # 创建或更新向量数据库
        # try:
        #     # 尝试加载现有数据库
        #     vector_db = FAISS.load_local(
        #         db_path,
        #         embeddings=embedding,
        #         allow_dangerous_deserialization=True
        #     )
        #     print("加载现有数据库成功，添加新文档...")
        #     vector_db.add_documents(all_documents)
        # except:
        print("创建新的向量数据库...")
        print(all_documents)
        vector_db = FAISS.from_documents(all_documents, embedding)
        
        # 保存数据库
        vector_db.save_local(db_path)
        print("成功保存向量数据库")
        
        # 测试搜索
        results = vector_db.similarity_search("黄金", k=7)
        print("\n测试搜索结果:")
        for i, doc in enumerate(results, 1):
            print(f"\n文档 {i}:")
            print(f"来源: {doc.metadata['source']}")
            print(f"内容: {doc.page_content[:100]}...")
        
    except Exception as e:
        print(f"处理过程中出错: {e}")


# 配置参数
directory_path = "test_file"  # 文档目录
doc_type = "technical"        # 文档类型
supported_extensions = [      # 支持的文件类型
    '.txt',
    '.md',
    '.pdf',
    '.json',
    '.csv'
]
    
# 执行导入
load_documents_from_directory(
    directory_path=directory_path,
    db_path="faiss_index",
    doc_type=doc_type,
    supported_extensions=supported_extensions
)

Created a chunk of size 2148, which is longer than the specified 1500
Created a chunk of size 4652, which is longer than the specified 1500



开始处理文件: test_file/金价向上、金店向下：金饰品牌加速关店.txt
文件大小: 4963 bytes
加载后的文档数量: 1
分割后的文本块数量: 2
成功加载文件: test_file/金价向上、金店向下：金饰品牌加速关店.txt

开始处理文件: test_file/Rank算法粗排实时主题表（dl_cpc.lego_dws_algo_sketchy_rank）.txt
文件大小: 13472 bytes
加载后的文档数量: 1
分割后的文本块数量: 4
成功加载文件: test_file/Rank算法粗排实时主题表（dl_cpc.lego_dws_algo_sketchy_rank）.txt

开始处理文件: test_file/test_pdf.pdf
文件大小: 10783628 bytes
加载后的文档数量: 724
分割后的文本块数量: 724
成功加载文件: test_file/test_pdf.pdf

总共处理了 3 个文件
总共获得 730 个文本块
创建新的向量数据库...
[Document(metadata={'source': 'test_file/金价向上、金店向下：金饰品牌加速关店.txt'}, page_content='与上一轮金价上涨期间，黄金投资、金饰消费均出现增长不同的是，如今金价越涨越高，黄金首饰却卖不动了。\n\n世界黄金协会近日公布的一组数据显示，过去一年全球金饰消费同比下降11%，其中，中国市场2024年金饰消费量同比下降24%。\n\n行业内多数金店对此感同身受，周大福、周生生、老凤祥过去一年出现销量、业绩的双下滑。发生了什么？未来金饰需求会有改善吗？\n\n金价上涨，金饰难卖\n\n在全球央行“购金热”的推动下，金价持续上涨，2月25日现货黄金一度站上2940美元/盎司，今年以来累计涨幅超过11%，时间略微拉长，自2024年以来，现货黄金累计涨幅超42%。\n\n黄金首饰兼具消费与投资属性，在上一轮“黄金大牛市”中，消费者对黄金首饰的购买量持续上涨。中国黄金协会数据显示，2010年我国黄金首饰需求357吨，随后一路上涨，到2013年首饰用金超716吨，需求量几乎翻了一倍。\n\n2024年国际金价全年涨幅超25%，但我国黄金市场的首饰用金需求却逆向而行，比上一年减少了151吨。\n\n具体到金饰

In [55]:
#测试txt文件向量化

from langchain.vectorstores import FAISS
from langchain.embeddings import OllamaEmbeddings
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter

# 设置嵌入模型
embedding = OllamaEmbeddings(
    model="bge-m3:latest",
    base_url="http://localhost:11434"
)

# 加载文档
loader = TextLoader('test_file/金价向上、金店向下：金饰品牌加速关店.txt')  # 替换为你的文档路径
documents = loader.load()

# 分割文档
# 根据文档类型选择参数
splitter_config = splitter_configs.get("news", splitter_configs["general"])
print(splitter_config)

# 创建分隔器
text_splitter = CharacterTextSplitter(**splitter_config)

texts = text_splitter.split_documents(documents)
print(texts)

# 创建向量数据库
vector_db = FAISS.from_documents(texts, embedding)

# 保存到本地（注意路径）
vector_db.save_local("faiss_index")

{'chunk_size': 500, 'chunk_overlap': 50, 'separator': '\n'}
[Document(metadata={'source': 'test_file/金价向上、金店向下：金饰品牌加速关店.txt'}, page_content='与上一轮金价上涨期间，黄金投资、金饰消费均出现增长不同的是，如今金价越涨越高，黄金首饰却卖不动了。\n世界黄金协会近日公布的一组数据显示，过去一年全球金饰消费同比下降11%，其中，中国市场2024年金饰消费量同比下降24%。\n行业内多数金店对此感同身受，周大福、周生生、老凤祥过去一年出现销量、业绩的双下滑。发生了什么？未来金饰需求会有改善吗？\n金价上涨，金饰难卖\n在全球央行“购金热”的推动下，金价持续上涨，2月25日现货黄金一度站上2940美元/盎司，今年以来累计涨幅超过11%，时间略微拉长，自2024年以来，现货黄金累计涨幅超42%。\n黄金首饰兼具消费与投资属性，在上一轮“黄金大牛市”中，消费者对黄金首饰的购买量持续上涨。中国黄金协会数据显示，2010年我国黄金首饰需求357吨，随后一路上涨，到2013年首饰用金超716吨，需求量几乎翻了一倍。\n2024年国际金价全年涨幅超25%，但我国黄金市场的首饰用金需求却逆向而行，比上一年减少了151吨。'), Document(metadata={'source': 'test_file/金价向上、金店向下：金饰品牌加速关店.txt'}, page_content='2024年国际金价全年涨幅超25%，但我国黄金市场的首饰用金需求却逆向而行，比上一年减少了151吨。\n具体到金饰企业，周大福公布的经营数据显示，2024年4-9月，周大福珠宝中国内地同店销售额下滑25.4%，按件数算的销量减少了30%；中国香港及中国澳门的同店销售额下滑30.8%，销售件数同比减少33%。虽然毛利率有所提升，达到了31.4%，但周大福整体销售额同比下降20.4%至394亿港元。\n2024年10-12月，周大福同店销售跌幅收窄，但销售额同比降幅仍超过16%，同店销量同比下滑幅度仍接近30%。\n销量下滑带来的一个后果是存货周转天数变长，周大福财报数据显示，2024年9月底的存货周转天数超过400天，2023年初的存货周转天数则为300天左右。\n有金店店员

In [8]:
#测试pdf文件向量化

from langchain.vectorstores import FAISS
from langchain.embeddings import OllamaEmbeddings
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter

# 设置嵌入模型
embedding = OllamaEmbeddings(
    model="bge-m3:latest",
    base_url="http://localhost:11434"
)

# 加载文档
loader = PyPDFLoader('test_file/test_pdf.pdf')  # 替换为你的文档路径
documents = loader.load()

# 分割文档
# 根据文档类型选择参数
splitter_config = splitter_configs.get("technical", splitter_configs["general"])
print(splitter_config)

# 创建分隔器
text_splitter = CharacterTextSplitter(**splitter_config)

texts = text_splitter.split_documents(documents)
print(texts)

# 创建向量数据库
vector_db = FAISS.from_documents(texts, embedding)

# 保存到本地（注意路径）
vector_db.save_local("faiss_index")

{'chunk_size': 1500, 'chunk_overlap': 300, 'separator': '\n\n'}
[Document(metadata={'producer': 'Microsoft® Word 2010', 'creator': 'Microsoft® Word 2010', 'creationdate': '2022-09-13T10:57:58+08:00', 'moddate': '2022-09-13T10:57:58+08:00', 'source': 'test_file/test_pdf.pdf', 'total_pages': 724, 'page': 0, 'page_label': '1'}, page_content='-ShLü   \n \n蜜雪冰城股份有限公司 \n（郑州市金水区北三环南、文化路东瀚海北金商业中心 16004 室） \n首次公开发行股票 \n \n招股说明书 \n（申报稿） \n \n保荐人（主承销商） \n \n（广东省广州市黄埔区中新广州知识城腾飞一街 2 号 618 室）'), Document(metadata={'producer': 'Microsoft® Word 2010', 'creator': 'Microsoft® Word 2010', 'creationdate': '2022-09-13T10:57:58+08:00', 'moddate': '2022-09-13T10:57:58+08:00', 'source': 'test_file/test_pdf.pdf', 'total_pages': 724, 'page': 1, 'page_label': '2'}, page_content='招股说明书（申报稿） \n1-1-1 \n声明：公司的发行申请尚未得到中国证监会核准。本招股说明书（申报\n稿）不具有据以发行股票的法律效力，仅供预先披露之用。投资者应当以正式\n公告的招股说明书全文作为做出投资决定的依据。 \n本次发行概况 \n发行股票类型 人民币普通股（A 股） \n发行股数 本次拟公开发行股票数量不超过4,001 万股，不低于发行后总股本的\n10%。本次发行全部为新股发行，不涉及股东公开发售股份。 \n每股面值 人民币1.00 元 \n每股发行

In [25]:
#测试faiss搜索

from langchain.vectorstores import FAISS
from langchain.embeddings import OllamaEmbeddings

# 使用 Ollama 的嵌入模型
embedding = NormalizedOllamaEmbeddings(
    model="bge-m3:latest",  # 或其他你想使用的模型
    base_url="http://localhost:11434"  # 使用你的 Ollama 服务端口
)

# 创建或加载向量数据库
vector_db = FAISS.load_local(
    "faiss_index", 
    embeddings=embedding,
    allow_dangerous_deserialization=True  # 添加这个参数
)

# 语义搜索
query = "rank表是什么"
# 增加 k 和 score_threshold
docs_and_scores = vector_db.similarity_search_with_score(
    query,
    k=7,                    # 返回更多结果
    score_threshold=0.9     # 设置相似度阈值
)

print("\n=== 搜索结果 ===")
for i, (doc, score) in enumerate(docs_and_scores, 1):
    print(f"\n文档 {i} (相似度分数: {score}):")
    print(f"来源: {doc.metadata['source']}")
    print(f"内容: {doc.page_content[:200]}...")
    print("-" * 50)



=== 搜索结果 ===

文档 1 (相似度分数: 0.8046020865440369):
来源: test_file/Rank算法粗排实时主题表（dl_cpc.lego_dws_algo_sketchy_rank）.txt
内容: # Rank算法粗排实时主题表（dl_cpc.lego_dws_algo_sketchy_rank）描述

## 表名
Rank算法粗排实时主题表（dl_cpc.lego_dws_algo_sketchy_rank）

## 字段说明...
--------------------------------------------------


In [ ]:
#直接使用 LLM + 模板

from langchain.callbacks.base import BaseCallbackHandler
from langchain.llms import Ollama
from langchain.prompts import PromptTemplate

# 处理搜索结果
def get_answer(query, search_results):
    # 合并上下文
    context = "\n\n".join([doc.page_content for doc in search_results])
    
    # 生成完整提示
    final_prompt = prompt.format(
        context=context,
        question=query
    )
    
    # 获取回答
    for chunk in llm.stream(final_prompt):
        print(chunk, end="", flush=True)

# 创建 LLM
llm = Ollama(
    model="deepseek-r1:14b",
    base_url="http://localhost:11434",
)

# 创建提示模板
template = """基于以下信息回答问题：

背景信息：
{context}

问题：{question}

请给出详细的回答："""

prompt = PromptTemplate(
    template=template,
    input_variables=["context", "question"]
)

query = "rank表是什么"

get_answer(query, vector_db.similarity_search(query, k=7, score_threshold=0.9))

<think>
嗯，我现在要理解一下“rank表”到底是什么。根据提供的背景信息，这个表叫做“Rank算法粗排实时主题表”，位于数据库dl_cpc.lego_dws_algo_sketchy_rank里。

首先，“Rank”这个词通常是指排名，所以这里的表应该和排序有关。粗排可能意味着这是一个初步的排序过程，不是最终的结果。比如，在搜索引擎中，可能会有多个阶段的排序，先有一个快速的粗排来过滤掉明显不相关的结果，然后再进行更精确的细排。

实时主题表这部分说明这个表里的数据是关于实时主题的，并且会不断更新。可能涉及当前热门话题、用户行为变化等实时信息。这让我想到，在推荐系统或者内容分发网络中，实时的主题信息对排序算法很重要，因为它们会影响用户的兴趣和互动情况。

字段方面，虽然没有详细列出每个字段的信息，但通常会有记录ID（比如主题ID）、分数或权重、时间戳这些关键指标。记录ID可能用来标识不同的主题；分数或权重用于评估该主题的重要性或相关性；时间戳则记录了数据的更新时间，确保实时性的准确性。

用途方面，这个表应该被Rank算法用来进行粗排处理。Rank算法可能是基于机器学习或者统计模型的，用来对大量的主题进行初步筛选和排序，以便后续流程能够更高效地处理。比如，在新闻推荐系统中，先用粗排去掉明显不相关的新闻，再进行详细的内容匹配和排名。

总结一下，rank表应该是一个用于实时更新和管理的主题列表，存储了各个主题的ID、相关评分或权重以及时间信息，用于Rank算法在初步筛选和排序时参考。这样可以在处理大量数据时提高效率，并及时反映当前的热门话题或用户兴趣变化。
</think>

**Answer:**

“rank表”是指“Rank算法粗排实时主题表”，位于数据库`dl_cpc.lego_dws_algo_sketchy_rank`中。

### 主要特点：
1. **目的和用途**：该表用于存储与Rank算法相关的实时主题数据，记录每个主题的ID、相关评分或权重，以及时间戳信息。这些数据帮助Rank算法进行初步筛选和排序（即粗排），以便后续处理更高效地进行。

2. **结构**：
   - **字段**：包括但不限于`theme_id`（主题ID）、`score`（得分/权重）和`timestamp`（时间戳）。
   - **实时性**：表中的数据会不断更